In [1]:
# November 2022
# Feature engineering
# Violeta Berdejo-Espinola

In [2]:
# %pip install -r ../requirements.txt
# %pip install spacy

In [3]:
import pandas as pd
import polars as pl

In [4]:
df = pl.read_csv('../data/v2/pos_neg.csv', encoding='utf-8')

In [5]:
# x data

corpus_list = (
    df.select(
        (pl.col("title") + " " + pl.col("abstract")).alias("title_abstract")
    )
    .to_series()
    .to_list()
)
print(len(corpus_list))
corpus_list[0:10]
# print(f'instances per\n{df["label"].value_counts()}')

5550


['Anatomía de la agalla en ficus benjamina (moraceae) asociada a "thrips" (tubulifera: phlaeothripidae) Las agallas son reconocidas generalmente como crecimientos anormales de los tejidos afectados por insectos cuando estos realizan la deposición de los huevecillos o se alimentan de los tejidos de la planta. En Ficus benjamina la acción del thrips Gynaikothrips garitacambroneroi al alimentarse de los tejidos de la hoja, provoca una agalla que consiste en el doblamiento de la hoja. En este trabajo se analizó la ultraestructura de secciones de hojas sanas y hojas con agallas de F. benjamina mediante el uso de la microscopia electrónica de barrido. Se analizó la cantidad de estomas por área y no se determinó alteración significativa, aunque se observa menor cantidad de cera cuticular en la superficie del área afectada por la agalla. En la zona de la hoja afectada por G. garitacambroneroi se observó bacilos y hongos y huevecillos de otros organismos, aparentemente invasores. Además, en los

# feature engineering

# removing special characters, punctiation, and numbers

In [6]:
import re
import string

# function to enact regex substitution on a list of strings

def sub_all(regex, corpus_list, replacement=" "):
    return [regex.sub(replacement, col) for col in corpus_list]

# define regular expressions
re_citation = re.compile(r"\(.[^())]*\d{4}[^())]*\)")
re_tabfig = re.compile(r"\(\s?\w{1,7}[.]?\s?\d{1}\w?\s?\)")
re_digit_char = re.compile(r"\d+\w{,2}")
re_one_two_letter = re.compile(r"\b\w{1,2}\b")
re_new_line = re.compile(r"\n{1,}")
re_tab = re.compile(r"\t{1,}")
re_html = re.compile(r"</?\w+>")
re_alt_html = re.compile(r"<.*?>")
re_spacing = re.compile(r"\s{2,}")
re_fig = re.compile(r"(fig)")
re_table = re.compile(r"(cuadro)")

# punctuation translation
punctuation_text = string.punctuation + "¿±♂♀’”°´“×–…" + "\xad" + "\xa0"
translator = str.maketrans(punctuation_text, " " * len(punctuation_text))

# function to process text and output 'clean corpus'

def text_processing(corpus_list):
    output = [
        col.lower() if isinstance(col, str) else "" 
        for col in corpus_list
    ]
    
    output = sub_all(re_citation, output)
    output = sub_all(re_tabfig, output)
    output = sub_all(re_fig, output)
    output = sub_all(re_table, output)
    output = sub_all(re_digit_char, output)
    output = sub_all(re_one_two_letter, output)
    
    # translate punctuation
    output = [col.translate(translator) for col in output]
    
    output = sub_all(re_new_line, output)
    output = sub_all(re_tab, output)
    output = sub_all(re_html, output)
    output = sub_all(re_alt_html, output)
    
    # strip extra spaces inside strings
    output = [col.strip() for col in output]
    output = sub_all(re_spacing, output)

    # final cleanup: strip again
    output = [col.strip() for col in output]
    
    return output

# function to process text and output 'raw corpus'

def text_processing_raw(text):

    output = sub_all(re_html, text)
    output = sub_all(re_alt_html, output)
    
    return output

corpus_clean = text_processing(corpus_list)
corpus_clean_raw = text_processing_raw(corpus_list) 
corpus_clean_raw 

['Anatomía de la agalla en ficus benjamina (moraceae) asociada a "thrips" (tubulifera: phlaeothripidae) Las agallas son reconocidas generalmente como crecimientos anormales de los tejidos afectados por insectos cuando estos realizan la deposición de los huevecillos o se alimentan de los tejidos de la planta. En Ficus benjamina la acción del thrips Gynaikothrips garitacambroneroi al alimentarse de los tejidos de la hoja, provoca una agalla que consiste en el doblamiento de la hoja. En este trabajo se analizó la ultraestructura de secciones de hojas sanas y hojas con agallas de F. benjamina mediante el uso de la microscopia electrónica de barrido. Se analizó la cantidad de estomas por área y no se determinó alteración significativa, aunque se observa menor cantidad de cera cuticular en la superficie del área afectada por la agalla. En la zona de la hoja afectada por G. garitacambroneroi se observó bacilos y hongos y huevecillos de otros organismos, aparentemente invasores. Además, en los

# lemmatization

In [7]:
import spacy

MODEL = 'es_core_news_md'
spacy.cli.download(MODEL) 
nlp = spacy.load(MODEL, disable=['parser', 'ner'])

# def lemmatizer(text):
    
#     doc_list = []
#     for sentence in text: 
#         doc_list.append(" ".join([token.lemma_ for token in nlp(" ".join(sentence))]))
    
#     return doc_list

def lemmatizer(text_list):
    return [" ".join(token.lemma_ for token in nlp(sentence)) for sentence in text_list]


corpus_clean1 = lemmatizer(corpus_clean)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 5.1 MB/s eta 0:00:0000:0100:01



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


# removing stopwords 

In [8]:
import re
from stop_words import get_stop_words

def remove_stopwords(sentences):
    stopwords = set(get_stop_words('spanish'))
    cleaned_sentences = []

    for sentence in sentences:
        cleaned_sentence = " ".join([
            word for word in sentence.split()
            if re.sub(r'\W+', '', word.lower()) not in stopwords and not word.isdigit()
        ])
        cleaned_sentences.append(cleaned_sentence)

    # Check if stopwords were removed
    all_words = ' '.join(cleaned_sentences).split()
    if any(word in stopwords for word in all_words):
        print('Stopwords not excluded from vocabulary.')
    else:
        print('Stopwords excluded from vocabulary.')

    # Check if numbers were removed
    if any(word.isdigit() for word in all_words):
        print('Numbers not excluded from vocabulary.')
    else:
        print('Numbers excluded from vocabulary.')

    return cleaned_sentences

corpus_clean2 = remove_stopwords(corpus_clean1)

Stopwords excluded from vocabulary.
Numbers excluded from vocabulary.


In [9]:
corpus_clean_raw

['Anatomía de la agalla en ficus benjamina (moraceae) asociada a "thrips" (tubulifera: phlaeothripidae) Las agallas son reconocidas generalmente como crecimientos anormales de los tejidos afectados por insectos cuando estos realizan la deposición de los huevecillos o se alimentan de los tejidos de la planta. En Ficus benjamina la acción del thrips Gynaikothrips garitacambroneroi al alimentarse de los tejidos de la hoja, provoca una agalla que consiste en el doblamiento de la hoja. En este trabajo se analizó la ultraestructura de secciones de hojas sanas y hojas con agallas de F. benjamina mediante el uso de la microscopia electrónica de barrido. Se analizó la cantidad de estomas por área y no se determinó alteración significativa, aunque se observa menor cantidad de cera cuticular en la superficie del área afectada por la agalla. En la zona de la hoja afectada por G. garitacambroneroi se observó bacilos y hongos y huevecillos de otros organismos, aparentemente invasores. Además, en los

In [10]:
# character length of each example before and after text preprocessing

each_example_len_1 = []
for each_example in corpus_list:
    each_example_len_1.append(sum(map(len, each_example)))

each_example_len_2 = []
for each_example in corpus_clean2:
    each_example_len_2.append(len(each_example))

lens = pd.DataFrame({"len_before_processing":each_example_len_1,
                    "len_after_processing":each_example_len_2})
lens

,len_before_processing,len_after_processing
0,1172,846
1,405,274
2,1404,1011
3,2013,1339
4,1234,892
...,...,...
5545,1430,1062
5546,1444,998
5547,1489,1078
5548,1907,1423


In [11]:
# creating lists of pos and neg instances

pos = df.filter(
    pl.col("label") == 1
    )
neg = df.filter(
    pl.col("label") == 0
    )
len(pos), len(neg) 

(44, 5506)

In [12]:
pos

title,abstract,body,journal,pub_year,abstract_length,language,label
str,str,str,str,i64,i64,str,i64
"""Influencia del rolado selectiv…","""El rolado selectivo de baja in…","""introducción el chaco occident…","""Ecología austral""",2015,1446,"""es""",1
"""Manejo del conflicto entre car…","""El conflicto entre carnívoros …","""introducción la conservaci ón …","""Mastozoología neotropical""",2017,1671,"""es""",1
"""Efecto de la restauración ecol…","""Los bosques templados del Parq…","""artículos de investigación ef…","""Madera y bosques""",2010,1627,"""es""",1
"""La conservación de mamíferos m…","""Resumen La fauna es un element…","""La fauna, y en particular los …","""Revista mexicana de biodiversi…",2018,1166,"""es""",1
"""El fuego favorece la invasión …","""A nivel mundial, se ha reporta…","""introducción los incendios for…","""Revista chilena de historia na…",2010,1771,"""es""",1
…,…,…,…,…,…,…,…
"""Erradicación de especies invas…","""El manejo de especies invasora…","""introducción el manejo de las …","""Mastozoología neotropical""",2016,1743,"""es""",1
"""Aplicación de la ley para el c…","""El tráfico ilegal es uno de lo…","""aplicación de la ley para el c…","""Therya""",2011,1490,"""es""",1
"""Recuperación de la colonia de …","""En la cueva de Cueva de Ágreda…","""El pueblo de Cueva de Ágreda (…","""Barbastella""",2012,1059,"""es""",1


In [13]:
# y data

y = (
    df.select(
        pl.col('label')
        )
    .to_series()
    .to_list()
    )
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
# data imbalance

len(pos)/len(corpus_clean)*100

0.7927927927927928

In [14]:
# save data to disk - serialise python object to bytes

import mpu

mpu.io.write('../data/v2/pos.pickle', pos)
mpu.io.write('../data/v2/neg.pickle', neg)

mpu.io.write("../data/v2/corpus_lemmatized.pickle", corpus_clean)

mpu.io.write("../data/v2/corpus_clean.pickle", corpus_clean2)
mpu.io.write("../data/v2/corpus_raw.pickle", corpus_clean_raw)

mpu.io.write("../data/v2/y.pickle", y)

lens.to_csv('../results/preprocessing/v2/diff_word_length_after_preprocessing.csv')

----------------------------------------------------------------------------------------------------------